In [10]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch

### Load Data

In [11]:
df = pd.read_csv("out.csv")
df.head()

,Date,Open,High,Low,Close,Volume
0,2022-08-04 05:50:00,23780.0,23780.0,23198.0,23198.0,84.921
1,2022-08-04 05:51:00,23228.2,23900.0,23198.0,23900.0,84.941
2,2022-08-04 05:52:00,23852.2,23898.0,23198.0,23898.0,132.174
3,2022-08-04 05:53:00,23898.0,23900.0,23198.0,23228.2,74.257
4,2022-08-04 05:54:00,23520.4,23946.0,23198.0,23653.4,83.717


### Prepare data

Here we take the data from source (in this case csv), then add/remove columns we need in order for the model to properly work

In [12]:
import helpers.process_data as process_data

# Initiate scaler
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

## process data

# number of rows in the future
# in this case we want to predict 1 single row into the future (1 minute)
FUTURE_PRICE_ROWS=1

data = process_data.process_df(df, future_n_rows=FUTURE_PRICE_ROWS, scaler=scaler)
print(data.head())

# Change dataframe data to numpy array
data = data.to_numpy()

# Inverse scale
# print(scaler.feature_names_in_[0])
# print(data[0,:2])
# print(scaler.inverse_transform([data[0,:2]]))

      Close    Volume  future_close
0 -0.703865  0.058691      1.270997
1  1.270997  0.058971      1.265370
2  1.265370  0.720967     -0.618907
3 -0.618907 -0.090771      0.577263
4  0.577263  0.041816     -0.703865


### Split and load data

Split data and load them into dataloaders.

Dataloaders are a great way to abstract and handle data for training process, this way you don't have to work with matrices, but objects (dataloaders)

In [13]:
# split data
train_data, test_data = process_data.split_data(data, test_percent=5)
print(train_data.shape)
print(test_data.shape)


(9500, 3)
(500, 3)


In [14]:
# Dataset and DataLoader
from helpers.dataloaders import StockDataSet
from torch.utils.data import DataLoader
train_dataset = StockDataSet(train_data)
test_dataset = StockDataSet(test_data)

BATCH_SIZE = 2

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)

### Set up model

Here we instantiate the model. In this case we use an LSTM RNN model since it has better long-range dependency between each step in the sequence.

In [15]:
import models.recursive_nets as model

"""
variables explanation for model instantiation

- 2 input vectors
- 64 hidden neurons
- 60 steps for each sequence
- 1 single output vector
"""

RNNModel = model.LSTM(2, 64, 60, 1)
RNNModel

LSTM(
  (rnn): LSTM(2, 64, batch_first=True)
  (linear): Linear(in_features=3840, out_features=1, bias=True)
)

### Calculate accuracy function

This function is in charge of calculating the accuracy for the regression model. It goes through all the items in the provided dataloader. it returns MAPE (mean absolute percentage error) which is basically the error percentage. The idea is that this function returns lower numbers on each iteration.

In [16]:
def calculate_accuracy_error(model: nn.Module, data: DataLoader, device: str) -> float:
    """Returns MAPE accuracy error percentage for the given dataloader complete run
    
    Keyword arguments:
    model: pytorch module
    data: dataloader to iterate and calculate MAPE

    Return:
    MAPE: Error percentage
    """
    model.eval()
    
    APE = list()

    for batch, (X, Y) in enumerate(data):
        X, Y = X.to(device), Y.to(device)
        with torch.no_grad():
            out = model(X)
            out = out.squeeze(dim=0)
            APE.append(torch.mean(torch.abs((Y - out) / Y)))

    MAPE = torch.mean(torch.Tensor(APE)).detach().item()
    return MAPE

# sample
calculate_accuracy_error(RNNModel, test_dataloader, "cpu")

1.1692607402801514

### Training Loop.

This holds the main logic to iterate through each batch on the dataloader and run the forward and backward pass on the model so it learns

Training data is uploaded to [wandb.ai](https://wandb.ai/). In there you can visualize the results

In [17]:
import wandb as wdb
from tqdm.notebook import tqdm

device = "cpu"

# Training Loop
def train(model: nn.Module,
    train_data: DataLoader,
    test_data: DataLoader,
    optimizer: optim.Optimizer,
    loss_fn: nn.Module,
    epochs: int = 10,
    enable_wandb: bool = False) -> None:
    """
    Trains the RNN model for the specified number of epochs

    Inputs
    ------
    model: RNN model to train (should inherit from nn.Module)
    train_data: Iterable DataLoader
    test_data: Iterable DataLoader
    epochs: Number of epochs to train the model
    optiimizer: Optimizer function to use for each epoch
    loss_fn: Loss function to use
    """
    if enable_wandb:
        wdb.init(project="trading-bot")

    train_losses = list()
    train_accuracy_error = list()
    test_accuracy_error = list()
    model.to(device)

    with tqdm(range(epochs), unit="epoch") as t:
            for epoch in t:
                model.train()
                epoch_losses = list()
                for X, Y in train_data:
                    # # skip batch if it doesnt match with the batch_size
                    # if X.shape[0] != model.batch_size:
                    #     continue

                    # send tensors to device
                    X, Y = X.to(device), Y.to(device)

                    # 2. clear gradients
                    model.zero_grad()

                    loss = 0
                    # Internal loop for RNN to train on each step of seq_length
                    in_vector = X
                    out = model(in_vector)
                    l = loss_fn(out.squeeze(dim=0), Y)
                    # print(f'in: {in_vector}')
                    # print(f'{out}, res: {Y[:, c]}')

                    loss += l
                    # print(loss.detach().item() / X.shape[1])

                    # 4. Compte gradients gradients
                    loss.backward()

                    # 5. Adjust learnable parameters
                    # clip as well to avoid vanishing and exploding gradients
                    nn.utils.clip_grad_norm_(model.parameters(), 3)
                    optimizer.step()
                    epoch_losses.append(loss.detach().item() / X.shape[0])
                train_losses.append(torch.tensor(epoch_losses).mean())
                train_accuracy_error.append(calculate_accuracy_error(RNNModel, train_data, device))
                test_accuracy_error.append(calculate_accuracy_error(RNNModel, test_data, device))
                # print(f'=> epoch: {epoch + 1}, loss: {train_losses[epoch]}')
                if enable_wandb:
                    wdb.log({'loss': train_losses[-1], 
                            'train_accuracy_error': train_accuracy_error[-1],
                            'test_accuracy_error': test_accuracy_error[-1]})
                t.set_postfix(train_losses=f"{train_losses[-1]:>5f}", 
                    train_acc=f"{train_accuracy_error[-1]:.2f}%",
                    test_acc=f"{test_accuracy_error[-1]:.2f}%",
                )
                

### Train

Since this is a regression problem we are using MSE as the loss function, and Adam for the optimizer

In [18]:


loss_function = nn.MSELoss()
optimizer = optim.Adam(RNNModel.parameters(), lr=1e-2)
train(RNNModel, train_dataloader, test_dataloader, optimizer, loss_function, 10, True)

loss,█▆▅▅▅▄▄▃▃▃▃▂▁▁▁▂▁▁
test_accuracy_error,▆▄▄▄▃▁▁▁▁▄▄▃▅█▅▆▄▄
train_accuracy_error,▆▄▄▄▃▁▁▁▁▄▄▃▅█▅▆▄▄
loss,0.07638
test_accuracy_error,3.25018
train_accuracy_error,3.25018


  0%|          | 0/10 [00:00<?, ?epoch/s]

/Users/diego/.local/share/virtualenvs/trading-bot-Nm060_Ai/lib/python3.10/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


### Evaluate

Function to evaluate a single sequence. This just accepts a single sequence and returns the predicted value

In [53]:
from sklearn.base import TransformerMixin

# sample for getting a single item from a dataset object. 
# No need for a dataloader here since it is a single input/output vector 
# (tuple[tensor, tensor])
# train_dataset[0]

def eval_model(
        model: nn.Module, 
        sequence: tuple[torch.Tensor, torch.Tensor],
        scaler: TransformerMixin | None = None
    ) -> float:
    """eval model with a single row input tensor
    
    Keyword arguments:
    model: RNN model
    sequence: complete input sequence (seq_length x out_features)
    scaler (Optional): Optional scaler to return scaled value

    Return: Predicted value (could be scaled if scaler is provided) 
    """
    
    model.eval()

    # add 1 more dimension to simulate a batch
    X = sequence[0].unsqueeze(0)
    Y = sequence[1]

    with torch.no_grad():
        out = model(X)

    if isinstance(scaler, TransformerMixin):
        zero_tensor = torch.Tensor([[out, 0]])
        out_scaled = scaler.inverse_transform(zero_tensor)
        return out_scaled.squeeze()[0].item()

    return out.detach().item()


eval_model(RNNModel, test_dataset[0], scaler)




23731.875541566627

### Save model